In [1]:
using DrWatson
quickactivate(@__DIR__)

## Remove time element and traverse the latent space to create swarming


1. Instead of training on windows of data, train only with static clouds
2. Traverse latent space to create swarming movement?!
3. ...
4. Profit


In [30]:
using Flux, CSV, DataFrames, MLDataPattern, StatsBase, CUDA, PlotlyJS
using BSON: @save
using Flux.Data: DataLoader
using ProgressMeter: Progress, next!
using Flux: logitbinarycrossentropy



In [11]:
df = DataFrame(CSV.File("$(datadir())/exp_raw/data.csv"; types=Float32))

function normalise(M) 
    min = minimum(minimum(eachcol(M)))
    max = maximum(maximum(eachcol(M)))
    return (M .- min) ./ (max - min)
end

normalised = Array(df) |> normalise

data = permutedims(reshape(permutedims(normalised, [2,1]),3,300,:), [2,1,3])

axes(data)


(Base.OneTo(300), Base.OneTo(3), Base.OneTo(3452))

In [57]:
function create_vae()
    # Define the encoder and decoder networks
    encoder_features = Chain(
       # 300x3xb
        Conv((9,), 3 => 64, relu; stride = 1, pad = SamePad()),
        MaxPool((2,)),
       # 150x64xb
        Conv((5,), 64 => 32, relu; stride = 1, pad = SamePad()),
        MaxPool((2,)),
        # 75x32xb
        Conv((5,), 32 => 12, relu; stride = 1, pad = SamePad()),
        MaxPool((3,)),
        # 25x12xb
        Flux.flatten,
        Dense(300, 300, relu)
    ) |> gpu 
        
    encoder_μ = Chain(encoder_features, Dense(300, 300)) |> gpu
    encoder_logvar = Chain(encoder_features, Dense(300, 300)) |> gpu

    decoder = Chain(
        Dense(300,300, relu),
        (x -> reshape(x, 25, 12, :)),
        # 25x12xb
        ConvTranspose((5,), 12 => 32, relu; stride = 1, pad = SamePad()),
        Upsample(scale=3),
        # 75x32xb
        ConvTranspose((5,), 32 => 64, relu; stride = 1, pad = SamePad()),
        Upsample(scale=2),
        # 150x64xb
        ConvTranspose((9,), 64 => 3, relu; stride = 1, pad = SamePad()),
        Upsample(scale=2)
        # 300x3xb
    ) |> gpu
        
    return encoder_μ, encoder_logvar, decoder
end

create_vae (generic function with 1 method)

In [58]:
function vae_loss(encoder_μ, encoder_logvar, decoder, x, β, λ)
    batch_size = size(x)[end]
    @assert batch_size != 0
    # Forward propagate through mean encoder and std encoders
    μ = encoder_μ(x)
    logvar = encoder_logvar(x)
    # Apply reparameterisation trick to sample latent
    z = μ + gpu(randn(Float32, size(logvar))) .* exp.(0.5f0 * logvar)
    # Reconstruct from latent sample
    x̂ = decoder(z)
    # Negative reconstruction loss Ε_q[logp_x_z]
    logp_x_z = -logitbinarycrossentropy(x̂ , x, agg=sum) / batch_size
    # KL(qᵩ(z|x)||p(z)) where p(z)=N(0,1) and qᵩ(z|x) models the encoder i.e. reverse KL
    # The @. macro makes sure that all operates are elementwise
    kl_q_p = 0.5f0 * sum(@. (exp(logvar) + μ^2 - logvar - 1f0)) / batch_size
    # Weight decay regularisation term
    reg = λ * sum(x->sum(x.^2), Flux.params(encoder_μ, encoder_logvar, decoder))
    # We want to maximise the evidence lower bound (ELBO)
    elbo = logp_x_z - β .* kl_q_p
    # So we minimise the sum of the negative ELBO and a weight penalty
    return -elbo + reg
end

vae_loss (generic function with 1 method)

In [16]:
function save_model(encoder_μ, encoder_logvar, decoder, save_dir::String, epoch::Int)
    print("Saving model...")
    let encoder_μ = cpu(encoder_μ), encoder_logvar = cpu(encoder_logvar), decoder = cpu(decoder)
        @save joinpath(save_dir, "model-$epoch.bson") encoder_μ encoder_logvar decoder
    end
    println("Done")
end

save_model (generic function with 1 method)

In [61]:
function train(encoder_μ, encoder_logvar, decoder, dataloader, num_epochs, λ, β, optimiser, save_dir)
    # The training loop for the model
    trainable_params = Flux.params(encoder_μ, encoder_logvar, decoder)

    for epoch_num = 1:num_epochs
        acc_loss = 0.0
        progress_tracker = Progress(length(dataloader), 1, "Training epoch $epoch_num: ")
        for x_batch in dataloader
            
            # pullback function returns the result (loss) and a pullback operator (back)
            loss, back = Flux.pullback(trainable_params) do
                vae_loss(encoder_μ, encoder_logvar, decoder, x_batch |> gpu, β, λ)
            end
            # Feed the pullback 1 to obtain the gradients and update then model parameters
            gradients = back(1f0)
            Flux.Optimise.update!(optimiser, trainable_params, gradients)
            if isnan(loss)
                break
            end
            acc_loss += loss
            next!(progress_traCker; showvalues=[(:loss, loss)])
        end
        @assert length(dataloader) > 0
        avg_loss = acc_loss / length(dataloader)
        metrics = DataFrame(epoch=epoch_num, negative_elbo=avg_loss)
        CSV.write(joinpath(save_dir, "metrics.csv"), metrics, header=(epoch_num==1), append=true)
        if epoch_num % 10 == 0 
            save_model(encoder_μ, encoder_logvar, decoder, save_dir, epoch_num)
        end
    end
    println("Training complete!")
end

train (generic function with 1 method)

In [23]:
batch_size = 32;

dataloader = DataLoader(data, batchsize=batch_size, shuffle=true, partial=false)
encoder_μ, encoder_logvar, decoder = create_vae()


(Chain(Chain(Conv((9,), 3 => 64, relu, pad=4), MaxPool((2,)), Conv((5,), 64 => 32, relu, pad=2), MaxPool((2,)), Conv((5,), 32 => 12, relu, pad=2), MaxPool((3,)), flatten, Dense(300, 300)), Dense(300, 300)), Chain(Chain(Conv((9,), 3 => 64, relu, pad=4), MaxPool((2,)), Conv((5,), 64 => 32, relu, pad=2), MaxPool((2,)), Conv((5,), 32 => 12, relu, pad=2), MaxPool((3,)), flatten, Dense(300, 300)), Dense(300, 300)), Chain(Dense(300, 300, relu), #3, ConvTranspose((5,), 12 => 32, relu, pad=2), Upsample(:nearest, scale = 3), ConvTranspose((5,), 32 => 64, relu, pad=2), Upsample(:nearest, scale = 2), ConvTranspose((9,), 64 => 3, relu, pad=4), Upsample(:nearest, scale = 2)))

In [52]:
η = 0.0001
β = 1f0
λ = 0.01f0
num_epochs = 500
save_dir = "results"

"results"

In [62]:
train(encoder_μ, encoder_logvar, decoder, dataloader, num_epochs, λ, β, ADAM(η), save_dir)


LoadError: UndefVarError: progress_traCker not defined